# Web scraping en la web de fotocasa.es

A continuación, se programa la consulta en fotocasa.es de los datos usados en el trabajo.

Se extraerán las viviendas de segunda mano de Sevilla y entorno.

## Carga de librerías

In [17]:
from selenium.webdriver.common.keys import Keys # Introducir datos
from selenium import webdriver # Arrancar el navegador
from selenium.webdriver.common.action_chains import ActionChains # Manejar el teclado
import pandas as pd
import numpy as np
import re # Expresiones regulares
import time # Añadir retardos
from datetime import datetime

## Apertura del navegador y acceso a la web

* Abrimos el navegador
* Accedemos a la web de fotocasa.es
* Maximizamos la pantalla

In [18]:
# driver = webdriver.Edge()

# Configurar las opciones del navegador para abrir en modo incógnito
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
# Inicializar el WebDriver con las opciones configuradas
driver = webdriver.Chrome(executable_path = r'chromedriver', options=options)

driver.get('https://www.fotocasa.es/')
driver.maximize_window()

## Aceptar cookies

* Pulsamos en el botón de aceptar cookies para quitarnos la ventana emergente

In [19]:
driver.find_element_by_id('didomi-notice-agree-button').click()

## Buscador

* Indicamos Sevilla comarca

In [20]:
driver.find_element_by_class_name('sui-AtomInput-input').send_keys('Sevilla capital y entorno, Sevilla')

* Pulsamos en el botón buscar para que nos muestre todas las viviendas a la venta en España

In [21]:
time.sleep(1)
driver.find_element_by_xpath('//*[@id="App"]/div[1]/main/div[1]/div[1]/div/div/div/div[1]/div[2]/div[2]/form/div[2]/button').click()

## Consulta de enlaces

Uno de los parámetros que tendremos en cuenta es si la vivienda es de obra nueva o de segunda mano. Una forma sencilla para registrarlo es valernos de los filtros provistos en la web.

### Segunda mano

* Pulsamos en el desplegable que indica el tipo de construcción de la vivienda

In [22]:
time.sleep(5)
driver.find_element_by_xpath('//*[@id="App"]/div[1]/div[3]/div/div/div[2]/div[2]/div').click()

* Seleccionamos la opción de segunda mano

In [23]:
time.sleep(2)
driver.find_element_by_xpath('//*[@id="App"]/div[1]/div[3]/div/div/div[2]/div[2]/div/div[2]/div[1]/div/div/div[2]/div/div[1]/label').click()

* Pulsamos en el botón para aplicar el filtro

In [24]:
driver.find_element_by_xpath('//*[@id="App"]/div[1]/div[3]/div/div/div[2]/div[2]/div/div[2]/div[2]/button').click()

* Creo un *dataframe* donde guardaré las consultas llamado `viviendas`

In [25]:
viviendas = pd.DataFrame()

Después de visitar cierta cantidad de páginas, podría aparecer una ventana emergente (la primera vez ocurió en la hoja 181). Buscamos si existe el botón para cerrarla y, en tal caso, lo pulsaremos. Como no sé en qué momento aparece la ventana, incluiré la orden en varios puntos del código. El botón es este:

> <button id="firstButton" class="modal__button modal__button--secondary ">Ahora no</button>

In [26]:
def existe_boton():
    try:
        driver.find_element_by_id('firstButton')
    except:
        return False
    return True

Función para combinar las características de las viviendas

In [27]:
def combinar_tuplas(lista_tuplas):
    # Creamos un diccionario para agrupar las tuplas por el primer elemento
    diccionario = {}
    for tupla in lista_tuplas:
        clave = tupla[0]
        valor = tupla[1]
        if clave in diccionario:
            # Si la clave ya existe, concatenamos el valor
            diccionario[clave] += ', ' + valor
        else:
            # Si la clave no existe, creamos una nueva entrada en el diccionario
            diccionario[clave] = valor
    
    # Reconstruimos la lista con las nuevas tuplas combinadas
    nueva_lista_tuplas = [(clave, valor) for clave, valor in diccionario.items()]
    
    return nueva_lista_tuplas

* Función para quitar caracteres extraños antes de exportar los datos a Excel

In [28]:
# Definir una función para eliminar caracteres ilegales
def remove_illegal_characters(text):
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
    if isinstance(text, str):
        return ILLEGAL_CHARACTERS_RE.sub('', text)
    return text

* Entramos en la primera vivienda

In [29]:
try: # En algún caso he encontrado que la foto de la vivienda no está en un div, por lo que la ruta es ...div/a[1]
    driver.find_element_by_xpath('//*[@id="App"]/div[1]/div[3]/main/div/div[3]/section/article[1]/div[2]/a').click()
except:
    driver.find_element_by_xpath('//*[@id="App"]/div[1]/div[3]/main/div/div[3]/section/article[1]/div/a[1]').click()

PrimeraVivienda = True
time.sleep(5)

* Empezamos el bucle

In [31]:
while True:
    nombres = ['URL', 'FechaConsulta', 'Precio', 'Titulo', 'Municipio', 'Descripcion']
    datos = []

    # URL
    datos.append(driver.current_url)
    
    # Fecha consulta
    datos.append(datetime.now())

    # Precio
    precio = driver.find_element_by_class_name('re-DetailHeader-price').text
    datos.append(precio)

    # Título
    titulo = driver.find_element_by_class_name('re-DetailHeader-propertyTitle').text
    datos.append(titulo)

    # Municipio
    municipio = driver.find_element_by_class_name('re-DetailHeader-municipalityTitle').text
    datos.append(municipio)

    # Descripción
    try: # Si existe botón "Leer más..." lo pulsamos
        driver.find_element_by_class_name('sui-MoleculeCollapsible-btn').click()
    except:
        pass
    try: # Por si no existiese descripción
        descripcion = driver.find_element_by_class_name('fc-DetailDescription').text
    except:
        descripcion = None
    datos.append(descripcion)

    # Características
    listaCaracteristicas = []

    ## Hay iconos al principio
    try: # Por si no existiese habitación
        habitaciones = driver.find_element_by_class_name("re-DetailHeader-rooms").text
    except:
        habitaciones = None
    listaCaracteristicas.append(('Habitaciones', habitaciones))

    try: # Por si no existiese baños
        baños = driver.find_element_by_class_name("re-DetailHeader-bathrooms").text
    except:
        baños = None
    listaCaracteristicas.append(('Baños', baños))

    try: # Por si no existiese superficie
        superficie = driver.find_element_by_class_name("re-DetailHeader-surface").text
    except:
        superficie = None
    listaCaracteristicas.append(('Superficie', superficie))
    
    try: # Por si existiese superficie del terreno (sólo en fincas rústicas)
        superficieTerreno = driver.find_element_by_class_name("re-DetailHeader-surface_terrain").text
    except:
        superficieTerreno = None
    listaCaracteristicas.append(('SuperficieTerreno', superficieTerreno))

    try: # Por si no existiese planta
        planta = driver.find_element_by_class_name("floor").text
    except:
        planta = None
    listaCaracteristicas.append(('Planta', planta))

    ## Hay una tabla
    leyendas = driver.find_elements_by_class_name('re-DetailFeaturesList-featureLabel')
    caracteristicas = driver.find_elements_by_class_name('re-DetailFeaturesList-featureValue')
    for i in range(0, len(leyendas)):
        leyenda = leyendas[i].text
        if not(leyenda.endswith(':')): # Si la leyenda acaba en ':' no nos interesa el registro
            caracteristica = caracteristicas[i].text
            listaCaracteristicas.append((leyenda, caracteristica))

    ## Hay una lista
    leyendas = driver.find_elements_by_class_name('re-DetailExtras-listItem')
    for i in range(0, len(leyendas)):
        leyenda = leyendas[i].text
        if not(leyenda.endswith(':')): # Si la leyenda acaba en ':' no nos interesa el registro
            listaCaracteristicas.append((leyenda, 'Sí'))

    ## Guardamos las características quitando duplicadas
    LeyCar = combinar_tuplas(list(set(listaCaracteristicas)))
    nombres.extend([leyenda for (leyenda,j) in LeyCar])
    datos.extend([caracteristica for (i,caracteristica) in LeyCar])

    # Coordenadas
    i = 0 # Bajamos hasta el final de la página para que se cargue el mapa
    faltaMapa = True
    while (i < 20000) & faltaMapa:
        driver.execute_script('window.scrollTo(0, ' + str(i) +');')
        try: # Intentamos sacar los datos del mapa
            coordenadas = driver.find_element_by_xpath("//a[@title='Abre esta zona en Google Maps (se abre en una nueva ventana)']").get_attribute('href')
            patron = r'll=(-?\d+\.\d+),(-?\d+\.\d+)' # Patrón de expresión regular para encontrar los números
            coincidencias = re.search(patron, coordenadas) # Buscar coincidencias en la cadena usando el patrón
            if coincidencias: # Verificar si se encontraron coincidencias
                latitud = float(coincidencias.group(1))
                longitud = float(coincidencias.group(2))
                nombres.extend(['Latitud', 'Longitud'])
                datos.extend([latitud, longitud])
            faltaMapa = False
        except:
            i += 500
            time.sleep(0.75)

    # Creo el dataframe de esta vivienda
    vivienda = pd.DataFrame(datos, index = nombres).transpose()

    # Añado este dataframe al que tiene todas las viviendas
    viviendas = pd.concat([viviendas, vivienda], ignore_index=True)
    numero_filas = viviendas.shape[0]

    # Cada 10 viviendas imprime un mensaje
    if numero_filas % 10 == 0:
        print(f"Llevas {numero_filas} viviendas")
    else:
        if numero_filas % 5 == 0:
            print(numero_filas)

    # Cada 100 viviendas guarda una copia de los datos
    # Lo bajo a 30 para que sea al final de cada página
    if numero_filas % 30 == 0:
        # Obtener la fecha y hora actuales y formatearlas
        now = datetime.now().strftime('%Y%m%d%H%M%S')
        # Crear el nombre del archivo con la fecha y hora actuales
        file_name = f'SegundaMano-{now}.xlsx'
        # Guardar el DataFrame en un archivo Excel con el nombre generado
        try:
            viviendas.to_excel(file_name, index=False)
        except: # Hay algún carácter extraño
            # Aplicar la función a todas las columnas de texto en el DataFrame
            viviendas_cleaned = viviendas.applymap(remove_illegal_characters)
            viviendas_cleaned.to_excel(file_name, index=False, engine='openpyxl')

    # Siguiente vivienda
    driver.execute_script('window.scrollTo(0, 0);')
    try: # Voy a la siguiente vivienda
        if PrimeraVivienda: # No existe botón "Anterior"
            driver.find_element_by_xpath('//*[@id="App"]/div[1]/main/div[1]/div[2]/a').click()
            PrimeraVivienda = False
        else:
            driver.find_element_by_xpath('//*[@id="App"]/div[1]/main/div[1]/div[2]/a[2]').click()
        time.sleep(5)
    except: # Si no hay siguiente vivienda, salgo y voy a la siguiente página
        try:
            driver.find_element_by_xpath('//*[@id="App"]/div[1]/main/div[1]/div[1]/a').click()
        except:
            driver.find_element_by_xpath('/html/body/div[2]/div[1]/main/div[1]/div[1]/a').click()
        time.sleep(5)
        # Bajamos poco a poco
        i = 7500
        mismaPagina = True
        while (i < 20000) & mismaPagina:
            driver.execute_script('window.scrollTo(0, ' + str(i) +');')
            try: # Si encontramos el botón de pasar página lo pulsamos
                boton = driver.find_elements_by_css_selector('li.sui-MoleculePagination-item')[-1]
                time.sleep(1)
                print("Hay botón")
                if boton.text == '':
                    print("Voy a pulsarlo")
                    boton.click()
                    print("Lo he pulsado")
                    time.sleep(1)
                    mismaPagina = False
                    # break
                else :
                    print('No hay más páginas')
                    # Obtener la fecha y hora actuales y formatearlas
                    now = datetime.now().strftime('%Y%m%d%H%M%S')
                    # Crear el nombre del archivo con la fecha y hora actuales
                    file_name = f'SegundaMano-{now}.xlsx'
                    # Guardar el DataFrame en un archivo Excel con el nombre generado
                    viviendas.to_excel(file_name, index=False)
                    1/0
            except:
                i += 100
                time.sleep(1)
        time.sleep(5)
        driver.execute_script('window.scrollTo(0, 0);') # Extremo superior de la página
        time.sleep(1)
        # Página actual
        print(driver.current_url)
        # Me salto otra página
        time.sleep(5)
        # Bajamos poco a poco
        i = 7500
        mismaPagina = True
        while (i < 20000) & mismaPagina:
            driver.execute_script('window.scrollTo(0, ' + str(i) +');')
            try: # Si encontramos el botón de pasar página lo pulsamos
                boton = driver.find_elements_by_css_selector('li.sui-MoleculePagination-item')[-1]
                time.sleep(1)
                print("Hay botón")
                if boton.text == '':
                    print("Voy a pulsarlo")
                    boton.click()
                    print("Lo he pulsado")
                    time.sleep(1)
                    mismaPagina = False
                    # break
                else :
                    print('No hay más páginas')
                    # Obtener la fecha y hora actuales y formatearlas
                    now = datetime.now().strftime('%Y%m%d%H%M%S')
                    # Crear el nombre del archivo con la fecha y hora actuales
                    file_name = f'SegundaMano-{now}.xlsx'
                    # Guardar el DataFrame en un archivo Excel con el nombre generado
                    viviendas.to_excel(file_name, index=False)
                    1/0
            except:
                i += 100
                time.sleep(1)
        time.sleep(5)
        driver.execute_script('window.scrollTo(0, 0);') # Extremo superior de la página
        time.sleep(1)
        # Página actual
        print(driver.current_url)
        
        
        try: # En algún caso he encontrado que la foto de la vivienda no está en un div, por lo que la ruta es ...div/a
            driver.find_element_by_xpath('//*[@id="App"]/div[1]/div[3]/main/div/div[3]/section/article[1]/div[2]/a').click()
        except:
            driver.find_element_by_xpath('//*[@id="App"]/div[1]/div[3]/main/div/div[3]/section/article[1]/div/a[1]').click()
        time.sleep(5)
        PrimeraVivienda = True
        time.sleep(1)

Hay botón
Voy a pulsarlo
Lo he pulsado
https://www.fotocasa.es/es/comprar/viviendas/sevilla-provincia/sevilla-capital-y-entorno/l/6?constructionTypeIds=2
Hay botón
Voy a pulsarlo
Lo he pulsado
https://www.fotocasa.es/es/comprar/viviendas/sevilla-provincia/sevilla-capital-y-entorno/l/7?constructionTypeIds=2
55
Llevas 60 viviendas
65
Llevas 70 viviendas
75
Llevas 80 viviendas
Hay botón
Voy a pulsarlo
Lo he pulsado
https://www.fotocasa.es/es/comprar/viviendas/sevilla-provincia/sevilla-capital-y-entorno/l/8?constructionTypeIds=2
Hay botón
Voy a pulsarlo
Lo he pulsado
https://www.fotocasa.es/es/comprar/viviendas/sevilla-provincia/sevilla-capital-y-entorno/l/9?constructionTypeIds=2
85
Llevas 90 viviendas
95
Llevas 100 viviendas
105
Llevas 110 viviendas
Hay botón
Voy a pulsarlo
Lo he pulsado
https://www.fotocasa.es/es/comprar/viviendas/sevilla-provincia/sevilla-capital-y-entorno/l/10?constructionTypeIds=2
Hay botón
Voy a pulsarlo
Lo he pulsado
https://www.fotocasa.es/es/comprar/viviendas/sevi

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".re-DetailHeader-price"}
  (Session info: chrome=125.0.6422.77)


In [77]:
# Definir una función para eliminar caracteres ilegales
def remove_illegal_characters(text):
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
    if isinstance(text, str):
        return ILLEGAL_CHARACTERS_RE.sub('', text)
    return text

# Aplicar la función a todas las columnas de texto en el DataFrame
viviendas_cleaned = viviendas.applymap(remove_illegal_characters)

now = datetime.now().strftime('%Y%m%d%H%M%S')
file_name = f'SegundaMano-{now}.xlsx'
viviendas_cleaned.to_excel(file_name, index=False, engine='openpyxl')

In [72]:
PrimeraVivienda = True

In [75]:
PrimeraVivienda = False

In [74]:
vivienda

,URL,Precio,Titulo,Municipio,Descripcion,,Superficie,SuperficieTerreno,Planta,Habitaciones,Baños,Latitud,Longitud
0,https://www.fotocasa.es/es/comprar/vivienda/ma...,228.000 €,"Piso en venta en Valverde , Universidad - Mala...",Madrid Capital,GLOBALPISO MALASAÑA VENDE EN EXCLUSIVA apartam...,", Sí",43 m²,None,2ª Planta,1 hab.,1 baño,40.423157,-3.701483


In [38]:
# Abrir navegador

driver = webdriver.Edge()
driver.get('https://www.fotocasa.es/')
driver.maximize_window()

driver.find_element_by_id('didomi-notice-agree-button').click()

driver.find_element_by_class_name('sui-AtomInput-input').send_keys('Sevilla capital y entorno, Sevilla')

time.sleep(1.5)
driver.find_element_by_xpath('//*[@id="App"]/div[1]/main/div[1]/div[1]/div/div/div/div[1]/div[2]/div[2]/form/div[2]/button').click()

time.sleep(5)
driver.find_element_by_xpath('//*[@id="App"]/div[1]/div[3]/div/div/div[2]/div[2]/div').click()

time.sleep(2)
driver.find_element_by_xpath('//*[@id="App"]/div[1]/div[3]/div/div/div[2]/div[2]/div/div[2]/div[1]/div/div/div[2]/div/div[1]/label').click()

driver.find_element_by_xpath('//*[@id="App"]/div[1]/div[3]/div/div/div[2]/div[2]/div/div[2]/div[2]/button').click()

In [40]:
# Ir a una página

pagina = 29
primerSalto = 7500
i = primerSalto
while (i < 20000) & (pagina > 1):
    print(f'I: {i}')
    driver.execute_script('window.scrollTo(0, ' + str(i) +');')
    try: # Si encontramos el botón de pasar página lo pulsamos
        boton = driver.find_elements_by_css_selector('li.sui-MoleculePagination-item')[-1]
        time.sleep(1)
        if boton.text == '':
            print(f'Faltan {pagina-1} páginas')
            boton.click()
            time.sleep(5)
            pagina -= 1
            print(f'He pulsado el botón en: {i}')
            i = 0
            print(f'I vale ahora: {i}')
            # break
        else :
            print('No hay más páginas')
            1/0
    except:
        i += 100
        time.sleep(0.5)

import winsound

# Produce un sonido de beep
frequency = 500  # Frecuencia en Hertz
duration = 500   # Duración en milisegundos (1000 ms = 1 segundo)

winsound.Beep(frequency, duration)
winsound.Beep(1000, duration)
winsound.Beep(frequency, duration)
winsound.Beep(2000, duration)

I: 7500
Faltan 28 páginas
He pulsado el botón en: 7500
I vale ahora: 0
I: 0
Faltan 27 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 26 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 25 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 24 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 23 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 22 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 21 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 20 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 19 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 18 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 17 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 16 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 15 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 14 páginas
He pulsado el botón en: 0
I vale ahora: 0
I: 0
Faltan 13 pági

In [70]:
PrimeraVivienda = True